In [1]:
import yaml
import torch

from torch_geometric.loader import DataLoader

from models import SWEGNN, GCN
from data import TemporalGraphDataset

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

dataset, info = TemporalGraphDataset(node_features=config['node_features'],
                    edge_features=config['edge_features'],
                    **config['dataset_parameters']).load()

In [4]:
print(dataset[0])
print(info)

Data(x=[1268, 6], edge_index=[2, 2612], edge_attr=[2612, 8], y=[1268, 1], pos=[2, 1268])
{'num_static_node_features': 3, 'num_dynamic_node_features': 1, 'num_static_edge_features': 5, 'num_dynamic_edge_features': 1, 'previous_timesteps': 2}


In [5]:
num_train = int(len(dataset) * 0.8) # 80% train, 20% test

train_dataset = dataset[:num_train]
# train_loader = DataLoader(train_dataset) # batch_size=32, shuffle=True

test_dataset = dataset[num_train:]
# test_loader = DataLoader(test_dataset) # batch_size=32, shuffle=True

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
base_model_params = {
    'static_node_features': info['num_static_node_features'],
    'dynamic_node_features': info['num_dynamic_node_features'],
    'static_edge_features': info['num_static_edge_features'],
    'dynamic_edge_features': info['num_dynamic_edge_features'],
    'previous_timesteps': info['previous_timesteps'],
    'device': device,
}
lr_info = config['training_parameters']

In [7]:
def train(model, loss_func, optimizer):
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for graph in train_dataset:
            graph = graph.to(device)
            labels = graph.y

            optimizer.zero_grad()

            outputs = model(graph)
            loss = loss_func(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / num_train
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')


def test(model, loss_func):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for graph in test_dataset:
            graph = graph.to(device)
            labels = graph.y

            outputs = model(graph)
            loss = loss_func(outputs, labels)
            running_loss += loss.item()

    # Print validation statistics
    print(f'Validation Loss: {running_loss:.4f}')

In [9]:
gcn_params = config['GCN']
model = GCN(**gcn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 69.6285
Epoch [2/10], Training Loss: 14.9909
Epoch [3/10], Training Loss: 13.7423
Epoch [4/10], Training Loss: 13.2381
Epoch [5/10], Training Loss: 12.6015
Epoch [6/10], Training Loss: 12.3423
Epoch [7/10], Training Loss: 12.5150
Epoch [8/10], Training Loss: 12.3075
Epoch [9/10], Training Loss: 12.6755
Epoch [10/10], Training Loss: 11.9781


In [11]:
test(model, loss_func)

Validation Loss: 729.9463


In [ ]:
swe_gnn_params = config['SWEGNN']
model = SWEGNN(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()
train(model, loss_func, optimizer)

SWEGNN(
  (edge_encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): PReLU(num_parameters=1)
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): PReLU(num_parameters=1)
    )
  )
  (static_node_encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): PReLU(num_parameters=1)
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): PReLU(num_parameters=1)
    )
  )
  (dynamic_node_encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=False)
      (1): PReLU(num_parameters=1)
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=False)
      (1): PReLU(num_parameters=1)
    )
  )
  (gnn_processors): ModuleList(
    (0): SWEGNNProcessor(node_features=64, edge_features=64, K=8)
  )
  (gnn_activations): Sequentia

In [17]:
train(model, loss_func, optimizer)

torch.Size([2612, 320])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2612x320 and 262x128)

In [ ]:
test(model, loss_func)